In [ ]:
import path_ocpy


In [ ]:
from ocpy import DDPSolver, iLQRSolver
from ocpy import MPC
import ocp_hexacopter

import numpy as np


In [ ]:
ocp = ocp_hexacopter.ocp_hexacopter_unconstrained(constrain='barrier')
# ocp = ocp_hexacopter.ocp_hexacopter_unconstrained(constrain='penalty', mu=1e0)
# ocp = ocp_hexacopter.ocp_hexacopter()


In [ ]:
# solver = DDPSolver(ocp, init=False)
solver = iLQRSolver(ocp, init=False)


In [ ]:
# If needed, reset parameters and set solution guess.
n_x = ocp.get_n_x()
n_u = ocp.get_n_u()
T = ocp.get_T()
N = ocp.get_N()
t0 = ocp.get_t0()
x0 = ocp.get_x0()

t0 = 0.0
x0 = np.array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
solver.set_initial_condition(t0, x0)

T = 1.5
N = 50
solver.set_horizon(T, N)

solver.reset_guess()
us_guess = np.array([[1, 1, 1, 1, 1, 1] for i in range(N)])
solver.set_guess(us_guess=us_guess)

In [ ]:
solver.init_solver()

In [ ]:
# set hyperparameters.
solver.set_line_search_param(alpha_min=1e-4, r_alpha=0.8)
# solver.set_regularization_param(
#     gamma_init=1e-3, r_gamma=5.0, gamma_min=0.0, gamma_max=1e6
# )
solver.set_stop_tol(stop_tol=1e-5)
solver.set_max_iters(1000)


In [ ]:
# Model Predictive Control class
mpc = MPC(solver)


In [ ]:
# Initialize mpc.
mpc.init_mpc(enable_line_search=True)

In [ ]:
import matplotlib.pyplot as plt


mpc._solver._result['noi']

xs = mpc._solver._xs_opt
us = mpc._solver._us_opt

plt.plot(xs)
plt.show()

plt.plot(us)
plt.show()


In [ ]:
# Run mpc.
xs, us, ts = mpc.run(
    T_sim=10, sampling_time=0.005, max_iters_mpc=5,
    result=True, log=True, plot=True
)


In [ ]:
%matplotlib inline
# Visualize
from ocpy.animator import HexacopterAnimator
animator = HexacopterAnimator(mpc.get_log_directory(), ocp.get_ocp_name())
animator.generate_animation(save=False, skip_rate=10)


In [ ]:
import matplotlib.pyplot as plt

result = mpc.get_result()

noi_hist = result['noi_hist']
plt.plot(noi_hist)
plt.title('NOI hist')
plt.show()
